# OpenAI Responses Client Direct Usage Example

This notebook demonstrates direct `OpenAIResponsesClient` usage for structured response generation with OpenAI models.

**Features:**
- Function calling capabilities with custom business logic
- Structured output generation with Pydantic models
- Streaming and non-streaming response modes

**Prerequisites:**
- Set `OPENAI_API_KEY` and `OPENAI_RESPONSES_MODEL_ID` environment variables (optionally configure `OPENAI_BASE_URL` and `OPENAI_ORG_ID`)
- Install the `agent-framework` Python package and dependencies

In [ ]:
# Import Required Libraries
import os
from random import randint
from typing import Annotated

from agent_framework import ChatResponse
from agent_framework.exceptions import ServiceResponseException
from agent_framework.openai import OpenAIResponsesClient
from dotenv import load_dotenv
from pydantic import BaseModel, Field

load_dotenv(override=True)

REQUIRED_ENV_VARS = ["OPENAI_API_KEY", "OPENAI_RESPONSES_MODEL_ID"]
missing_env_vars = [var for var in REQUIRED_ENV_VARS if not os.environ.get(var)]
if missing_env_vars:
    raise OSError("Missing required environment variables: " + ", ".join(missing_env_vars))

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
OPENAI_RESPONSES_MODEL_ID = os.environ["OPENAI_RESPONSES_MODEL_ID"]
OPENAI_BASE_URL = os.environ.get("OPENAI_BASE_URL", "").strip()
OPENAI_ORG_ID = os.environ.get("OPENAI_ORG_ID", "").strip()


def build_responses_client() -> OpenAIResponsesClient:
    """Create an OpenAI Responses client using environment configuration."""
    return OpenAIResponsesClient(
        api_key=OPENAI_API_KEY,
        model_id=OPENAI_RESPONSES_MODEL_ID,
        base_url=OPENAI_BASE_URL or None,
        org_id=OPENAI_ORG_ID or None,
    )


def handle_service_exception(context: str, exc: ServiceResponseException) -> None:
    """Provide actionable guidance when OpenAI returns an error."""
    error_text = str(exc)
    print(f"{context} failed: {error_text}")
    if "404" in error_text:
        print(
            "Hint: The requested model may be unavailable. Verify "
            "OPENAI_RESPONSES_MODEL_ID matches an enabled Responses model."
            "enabled Responses model."
        )
    elif "401" in error_text:
        print("Hint: OPENAI_API_KEY may be invalid or missing. Regenerate the key and try again.")
    elif "429" in error_text:
        print(
            "Hint: The request hit rate or quota limits. Reduce throughput "
            "or request a higher rate limit."
        )


client = build_responses_client()

## Define Custom Tools

Create a simple weather tool that the agent can call to get weather information for different locations.

In [ ]:
def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    """Get the weather for a given location."""
    conditions = ["sunny", "cloudy", "rainy", "stormy"]
    temp = randint(10, 30)
    return f"The weather in {location} is {conditions[randint(0, 3)]} with a high of {temp}°C."

## Define Structured Output Format

Use Pydantic models to define the expected structure of the agent's response.

In [ ]:
class OutputStruct(BaseModel):
    """Structured output for weather information."""

    location: str
    weather: str

## Initialize the Client and Get Response

Create an OpenAI Responses Client and query for weather information with structured output.

In [ ]:
# Display the configuration being used for clarity
configured_base_url = getattr(client, "base_url", None)
if not configured_base_url or configured_base_url == "None":
    configured_base_url = "https://qredence-foundry.openai.azure.com/openai/v1"
print(f"Configured model: {client.model_id}")
print(f"Using base URL: {configured_base_url}")

# Define the user message
message = "What's the weather in Amsterdam and in Paris?"
print(f"User: {message}")

## Option 1: Streaming Response

Get a streaming response from the model, which provides real-time output as it's generated.

In [ ]:
# Get streaming response
try:
    response = await ChatResponse.from_chat_response_generator(
        client.get_streaming_response(message, tools=get_weather, response_format=OutputStruct),
        output_format_type=OutputStruct,
    )
    print(f"Assistant: {response.value}")
except ServiceResponseException as exc:
    handle_service_exception("Streaming response", exc)

## Option 2: Non-Streaming Response

Get a complete response in a single call without streaming.

In [ ]:
# Get non-streaming response
try:
    response = await client.get_response(message, tools=get_weather, response_format=OutputStruct)
    print(f"Assistant: {response.value}")
except ServiceResponseException as exc:
    handle_service_exception("Non-streaming response", exc)